In [2]:
import os
import re
import logging
from datetime import datetime
import numpy as np
import pandas as pd

In [42]:
dataframes = {}
dfs = {}
path = '../data/etl/e/'

for file_name in os.listdir(path):
    file_path = os.path.join(path, file_name)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        dataframes[file_name] = df
len(dataframes)

for key, df in dataframes.items():
    splitted = key.split()
    shop_name = splitted[0]
    cols_level_1 = list(df.iloc[0])
    cols_level_2 = list(df.iloc[1])
    combined_list = [
        cols_level_1[i] if not pd.isna(cols_level_1[i]) else cols_level_2[i]
        for i in range(len(cols_level_1))
    ]
    df.columns = combined_list
    df.drop([0, 1], inplace=True)
    if "Фамилия, имя Кассира" in combined_list:
        df = df[df["ОБЩАЯ ВЫРУЧКА (ПРИХОД)*"].notna()].copy()
        lst_of_cols = [
            "Дата",
            "ОБЩАЯ ВЫРУЧКА (ПРИХОД)*",
            "Фамилия, имя Кассира",
            "Остаток на конец дня*",
        ]
        df = df[lst_of_cols].copy()
        df.loc[:, "Магазин"] = shop_name
        df.rename(
            columns={
                "ОБЩАЯ ВЫРУЧКА (ПРИХОД)*": "Выручка",
                "Фамилия, имя Кассира": "ФИО",
            },
            inplace=True,
        )
        result = df.reset_index(drop=True)
        dfs[shop_name] = result
    elif "Фамилия Кассир №1" in combined_list:
        if "Фамилия Кассир №2 (ночь)" in combined_list:
            second_cashier = "Фамилия Кассир №2 (ночь)"
        else:
            second_cashier = "Фамилия Кассир №2"
        df = df[df["ОБЩАЯ ВЫРУЧКА (ПРИХОД)*"].notna()].copy()
        lst_of_cols_1 = [
            "Дата",
            "ОБЩАЯ ВЫРУЧКА (ПРИХОД)*",
            "Остаток на конец дня*",
            "Фамилия Кассир №1",
        ]
        lst_of_cols_2 = [
            "Дата",
            "ОБЩАЯ ВЫРУЧКА (ПРИХОД)*",
            "Остаток на конец дня*",
            second_cashier,
        ]
        df1 = df[lst_of_cols_1].copy()
        df2 = df[df[second_cashier].notnull()][lst_of_cols_2].copy()
        df1.rename(
            columns={
                "ОБЩАЯ ВЫРУЧКА (ПРИХОД)*": "Выручка",
                "Фамилия Кассир №1": "ФИО",
            },
            inplace=True,
        )
        df2.rename(
            columns={"ОБЩАЯ ВЫРУЧКА (ПРИХОД)*": "Выручка", second_cashier: "ФИО"},
            inplace=True,
        )
        result = pd.concat([df1, df2], ignore_index=True)
        result["Магазин"] = shop_name
        dfs[shop_name] = result
concatenated_df = pd.concat(dfs.values(), ignore_index=True)



# Use raw string for regex to avoid SyntaxWarning
concatenated_df["Магазин"] = concatenated_df["Магазин"].str.extract(
    r"(\d+)", expand=False
)
# Handle NaNs before converting to int
concatenated_df["Магазин"] = concatenated_df["Магазин"].fillna(0).astype(int)

In [45]:
concatenated_df.dtypes

Дата                     object
Выручка                  object
Остаток на конец дня*    object
ФИО                      object
Магазин                   int64
dtype: object

In [46]:
concatenated_df["Дата"] = pd.to_datetime(
        concatenated_df["Дата"], errors="coerce"
    ).dt.to_pydatetime()

/tmp/ipykernel_343171/2299148897.py:3: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ).dt.to_pydatetime()


In [47]:
concatenated_df

,Дата,Выручка,Остаток на конец дня*,ФИО,Магазин
0,2023-01-01,111410,61920,Казакова,5
1,2023-01-02,89300,28810,Минина,5
2,2023-01-03,112080,41710,Минина,5
3,2023-01-04,78810,19660,Казакова,5
4,2023-01-05,96980,35480,Казакова,5
...,...,...,...,...,...
5862,2025-01-03,97840,35057,Владимирова,3
5863,2025-01-04,39375,40392,Владимирова,3
5864,2025-01-05,126397,53722,Владимирова,3
5865,2025-01-07,63756,63332,Владимирова,3


In [12]:




    concatenated_df["Дата"] = pd.to_datetime(
        concatenated_df["Дата"], format="%d.%m.%Y", errors="coerce"
    ).dt.to_pydatetime()

    # Rename the columns
    concatenated_df.rename(
        columns={
            "Дата": "date",
            "Выручка": "revenue",
            "Остаток на конец дня*": "balance",
            "ФИО": "seller_name",
            "Магазин": "store_id",
        },
        inplace=True,
    )

    concatenated_df["balance"] = pd.to_numeric(
        concatenated_df["balance"], errors="coerce"
    )
    concatenated_df["revenue"] = pd.to_numeric(
        concatenated_df["revenue"], errors="coerce"
    )

    # Clean 'seller_name' column
    concatenated_df["seller_name"] = (
        concatenated_df["seller_name"].str.strip().fillna("Неизвестно")
    )

    return concatenated_df


NameError: name 'dfs' is not defined